In [1]:
import pandas as pa
import os
import csv
import tqdm
import string
import re

### Merge data from mt5_intersection and mt5_factver to one file

In [14]:
files_list = os.listdir('.')
files = []
for file in files_list:
    if('mt5_factver' in file):
        files.append(file)
print("Files:")
print(files)

Files:
['mt5_factver_inference_mt5-small.csv']


In [15]:
title = ""
for file in files:
    with open(file, mode ='r') as fin:
        csvData = csv.reader(fin)
        for lineData in csvData:
            print(lineData)
            break

['input_texts', 'lang', 'ref_text', 'pred_text']


In [16]:
writeData = dict()
titleSet = set()
for file in tqdm.tqdm(files):
    with open(file, mode ='r') as fin:
        csvData = csv.reader(fin)
        for i,lineData in enumerate(csvData):
            if i == 0:
                continue
            lineTitle = lineData[0].split(' ')[1]
            if lineTitle in titleSet:
                writeData[lineTitle].append([lineTitle, lineData[1],lineData[3]])
            else:
                writeData[lineTitle] = [[lineTitle, lineData[1],lineData[3]]]
                titleSet.add(lineTitle)
with open('factver.csv', 'w', encoding='utf-8') as csvfile:
    csvwriter = csv.writer(csvfile)
    for key in tqdm.tqdm(writeData.keys()):
        csvwriter.writerows(writeData[key])            

100%|██████████████████████████████████████████| 4669/4669 [00:00<00:00, 70437.36it/s]


### Fix pred and ref text in mt5_intersection

In [2]:
base_dir = 'inference/'
out_dir = 'inference_fixed/'
files_list = os.listdir(base_dir)
files = []
for file in files_list:
    if('mt5_intersection' in file):
        files.append(file)
print("Files:")
print(files)

Files:
['mt5_intersection_mt5-smalldbb2bec7.csv', 'mt5_intersection_mt5-small083cce06.csv', 'mt5_intersection_mt5-smallba52c9de.csv', 'mt5_intersection_mt5-smallac1104aa.csv']


In [3]:
for file in files:
    with open(os.path.join(base_dir,file), mode ='r') as fin:
        csvData = csv.reader(fin)
        for lineData in csvData:
            print(lineData)
            break

['input_texts', 'lang', 'ref_text', 'pred_text']
['input_texts', 'lang', 'ref_text', 'pred_text']
['input_texts', 'lang', 'ref_text', 'pred_text']
['input_texts', 'lang', 'ref_text', 'pred_text']


In [4]:
for file in tqdm.tqdm(files):
    fin = open(os.path.join(base_dir, file), mode ='r')
    fout = open(os.path.join(out_dir, file), mode = 'w',encoding='utf-8')
    csvWriter = csv.writer(fout)
    csvData = csv.reader(fin)
    for i,lineData in enumerate(csvData):
        if i == 0:
            csvWriter.writerow(lineData)
            continue
        refText = lineData[2].lower()
        for punc in string.punctuation:
            refText = refText.replace(punc,' ')
        predText = re.sub(r' <sep>',r'', lineData[3][5:].lower())
        for punc in string.punctuation:
            predText = predText.replace(punc,' ')
        csvWriter.writerow([lineData[0],lineData[1],refText,predText])
    fin.close()
    fout.close()

100%|██████████| 4/4 [00:00<00:00, 47.84it/s]


In [7]:
fout = open("final_article.csv", mode = 'w',encoding='utf-8')
csvWriter = csv.writer(fout)
for i,file in tqdm.tqdm(enumerate(files)):
    with open(os.path.join(out_dir,file), mode ='r') as fin:
        csvData = csv.reader(fin)
        for j,lineData in enumerate(csvData):
            if i == 0 and j == 0:
                csvWriter.writerow(lineData)
            elif j == 0:
                continue
            csvWriter.writerow(lineData)            
fout.close()

4it [00:00, 130.30it/s]
